# Batched Wikidata Scraper for News Media

This notebook retrieves detailed information about news and media organisations from **Wikidata**, including their social media profiles (Twitter, Facebook, Instagram, TikTok, YouTube) and organisational metadata such as headquarters, country, language, inception date, owner, and parent organisation.

To respect the limits of the SPARQL endpoint and avoid timeouts, the scraper processes the Q-IDs in batches and pauses between requests as configured.  Results are saved both in raw and deduplicated form as CSV and JSON files.


## Optional: Mount Google Drive

If you are running this notebook on Google Colab and wish to save the output files to your Drive, run the following cell.  On other platforms, you can omit this step or modify `OUTPUT_DIR` in the configuration section below to a local path.


In [ ]:

# Mount Google Drive (only required on Google Colab)
from google.colab import drive

drive.mount('/content/drive')


## Configuration

Set the output directory and batching parameters here.  The `BATCH_SIZE` determines how many Q-IDs are requested per SPARQL query; `SLEEP_SEC` controls the pause between batches to avoid overloading the endpoint.  When running on Colab, ensure that `OUTPUT_DIR` points to a directory you have write permissions for (for example, a folder within your Drive).


In [ ]:

import os

# Directory where CSV/JSON files will be saved.  Change this as needed.
OUTPUT_DIR = "/content/drive/MyDrive/YouTube/SPARQL/"

# Number of Q-IDs to request per SPARQL query.  A smaller batch size may reduce the chance of timeouts.
BATCH_SIZE = 25

# Seconds to sleep between batch requests.  Increase this if you encounter rate limiting or need to throttle requests.
SLEEP_SEC = 0.0

# Create the output directory if it does not already exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Files will be written to: {OUTPUT_DIR}")
print(f"BATCH_SIZE = {BATCH_SIZE}   |   SLEEP = {SLEEP_SEC} s")


## Helper functions

The following functions assist with making requests to the SPARQL endpoint, safely extracting values from `pandas` rows, and building the `VALUES` block of SPARQL queries for batched requests.


In [ ]:

import requests
import pandas as pd
import time
import json
import math
import textwrap
from urllib.parse import urlparse

# SPARQL endpoint and headers for Wikidata queries
ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {"User-Agent": "NewsYTMapping/3.0 (Colab; contact: your_email@example.com)"}


def run_query(query: str) -> pd.DataFrame:
    """Execute a SPARQL query and return the results as a DataFrame.

    Raises an HTTP error if the request fails.
    """
    response = requests.get(
        ENDPOINT,
        params={"query": query, "format": "json"},
        headers=HEADERS,
        timeout=60,
    )
    response.raise_for_status()
    return pd.DataFrame([
        {k: v["value"] for k, v in b.items()}
        for b in response.json()["results"]["bindings"]
    ])


def safe_get(series: pd.Series, key: str) -> str:
    """Safely extract a value from a pandas Series, returning an empty string if missing or NaN."""
    val = series.get(key, "")
    return val if (pd.notna(val) and val != "") else ""


def build_values_block(qids: list) -> str:
    """Build a SPARQL VALUES clause from a list of Q-IDs."""
    inside = " ".join(f"wd:{qid}" for qid in qids)
    return f"VALUES ?item {{ {inside} }}"


## Collect news-media Q-IDs

This step retrieves the Wikidata identifiers (Q-IDs) for all items that are classified as news/media organisations (instances or subclasses of `Q1193236`) and have an official website (`P856`).  The Q-IDs are fetched in chunks to avoid timeouts.


In [ ]:

# Collect Q-IDs for news/media organisations with an official website
items = []
chunk_size = 10000
offset = 0
print("Gathering Q-IDs (news media) with an official website …")
while True:
    query_chunk = f"""
    SELECT ?item WHERE {{
      ?item wdt:P856 [] .
      ?item wdt:P31/wdt:P279* wd:Q1193236 .
    }} LIMIT {chunk_size} OFFSET {offset}
    """
    try:
        df_chunk = run_query(query_chunk)
    except Exception as e:
        print(f"⚠️  Chunk offset {offset} failed: {e}")
        offset += chunk_size
        time.sleep(SLEEP_SEC)
        continue
    if df_chunk.empty:
        break
    # Extract Q-IDs from the full URLs
    items.extend(df_chunk["item"].str.split("/").str[-1])
    offset += chunk_size
    print(f"  – collected {len(items):,}")
    time.sleep(SLEEP_SEC)

print(f"✓ Total news-media Q-IDs: {len(items):,}
")


## Batched retrieval of organisation data

With the list of Q-IDs in hand, the notebook builds a SPARQL query for each batch.  Each query requests a variety of social media properties and organisation metadata, using `OPTIONAL` clauses so that missing data does not exclude an organisation from the results.  Between batches, the script pauses for `SLEEP_SEC` seconds.


In [ ]:

# Retrieve organisation data in batches
basic_rows = []
full_rows = []
batches = math.ceil(len(items) / BATCH_SIZE)

for batch_index in range(batches):
    batch_qids = items[batch_index * BATCH_SIZE : (batch_index + 1) * BATCH_SIZE]
    values_clause = build_values_block(batch_qids)
    sparql_batch = textwrap.dedent(f"""
    SELECT ?item ?itemLabel ?official_website ?twitter ?facebook ?instagram ?tiktok
           ?youtube_channel ?youtube_handle ?youtube_playlists ?headquartersLabel
           ?countryLabel ?languageLabel ?inception ?ownerLabel ?parentOrgLabel ?subscribers
    WHERE {{
      {values_clause}
      ?item wdt:P856 ?official_website .
      OPTIONAL {{ ?item wdt:P2002 ?twitter. }}
      OPTIONAL {{ ?item wdt:P2013 ?facebook. }}
      OPTIONAL {{ ?item wdt:P2003 ?instagram. }}
      OPTIONAL {{ ?item wdt:P7085 ?tiktok. }}
      OPTIONAL {{ ?item wdt:P2397 ?youtube_channel. }}
      OPTIONAL {{ ?item wdt:P11245 ?youtube_handle. }}
      OPTIONAL {{ ?item wdt:P4300 ?youtube_playlists. }}
      OPTIONAL {{ ?item wdt:P159 ?headquarters. }}
      OPTIONAL {{ ?item wdt:P17 ?country. }}
      OPTIONAL {{ ?item wdt:P407 ?language. }}
      OPTIONAL {{ ?item wdt:P571 ?inception. }}
      OPTIONAL {{ ?item wdt:P127 ?owner. }}
      OPTIONAL {{ ?item wdt:P749 ?parentOrg. }}
      OPTIONAL {{
        ?item p:P2397 ?ytStmt .
        ?ytStmt ps:P2397 ?youtube_channel .
        ?ytStmt pq:P3744 ?subscribers .
      }}
      SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en".
        ?item rdfs:label ?itemLabel .
        ?headquarters rdfs:label ?headquartersLabel .
        ?country rdfs:label ?countryLabel .
        ?language rdfs:label ?languageLabel .
        ?owner rdfs:label ?ownerLabel .
        ?parentOrg rdfs:label ?parentOrgLabel .
      }}
    }}
    """)
    try:
        df = run_query(sparql_batch)
    except Exception as e:
        print(f"⚠️  Batch {batch_index+1}/{batches} failed: {e} – skipped.")
        time.sleep(SLEEP_SEC)
        continue

    if df.empty:
        time.sleep(SLEEP_SEC)
        continue

    # Collect full records
    full_rows.extend(df.to_dict("records"))

    # Collect basic records (one per item) for a simpler mapping
    for qid in batch_qids:
        sub = df[df["item"].str.endswith(qid)]
        if sub.empty:
            continue
        r0 = sub.iloc[0]
        youtube_id = safe_get(r0, "youtube_channel")
        yt_url = f"https://www.youtube.com/channel/{youtube_id}" if youtube_id else ""
        basic_rows.append({
            "item": qid,
            "name": safe_get(r0, "itemLabel"),
            "news_domain": safe_get(r0, "official_website"),
            "youtube_url": yt_url,
        })

    # Progress logging
    if (batch_index + 1) % 10 == 0 or batch_index + 1 == batches:
        done = min((batch_index + 1) * BATCH_SIZE, len(items))
        print(f"✓ batches {batch_index+1}/{batches}  |  items {done:,}/{len(items):,}")
    time.sleep(SLEEP_SEC)


## Deduplicate and save results

After retrieving data for all batches, we create DataFrames from the collected records, deduplicate them by `item` (Q-ID), and write four files to the output directory:

- **news_domain_youtube_raw** – one row per item per record (may include duplicates);
- **news_domain_youtube_unique** – unique items with basic data (one row per organisation);
- **all_info_raw** – raw full data with all retrieved fields;
- **all_info_unique** – unique full data (one row per organisation).

Each file is saved in both CSV and JSON formats and can be uploaded to GitHub alongside this notebook for further analysis or reuse.


In [ ]:

# Convert the collected data to DataFrames
df_basic_raw = pd.DataFrame(basic_rows)
df_full_raw  = pd.DataFrame(full_rows)

# Deduplicate by Q-ID
df_basic_unique = df_basic_raw.drop_duplicates(subset=["item"])
df_full_unique  = df_full_raw.drop_duplicates(subset=["item"])

# Function to save a DataFrame to CSV and JSON
def save_df(df: pd.DataFrame, stem: str):
    csv_path  = os.path.join(OUTPUT_DIR, f"{stem}.csv")
    json_path = os.path.join(OUTPUT_DIR, f"{stem}.json")
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", force_ascii=False, indent=2)
    print(f"  • {stem}  ({len(df):,} rows)")

print("
Writing files …")
save_df(df_basic_raw , "news_domain_youtube_raw")
save_df(df_basic_unique , "news_domain_youtube_unique")
save_df(df_full_raw  , "all_info_raw")
save_df(df_full_unique , "all_info_unique")

print("
✓ Finished. Files saved to the output directory above.")


## Summary

This notebook demonstrates how to perform a batched retrieval of news/media organisations from Wikidata, capturing a wide range of social media and organisational metadata.  By batching requests, respecting the SPARQL endpoint’s limits and adding appropriate pauses, it is possible to collect detailed datasets for thousands of entities.

Feel free to modify the SPARQL query to include other properties or to tailor the dataset to your needs.
